In [66]:
import os , getpass

from gradio_client.utils import dict_or_str_to_json_file
from langchain.chat_models import init_chat_model
from pydantic import BaseModel, Field
from langchain_core.tools import tool
from langgraph.graph import add_messages
from typing import TypedDict, Annotated

In [3]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [7]:
if not os.environ.get("GITHUB_TOKEN"):
    os.environ["GITHUB_TOKEN"] = getpass.getpass("Enter API key for Github: ")

In [41]:
llm = init_chat_model("openai:gpt-4o-mini")

In [42]:
map_api_template = {
    "golang_api_echo":"golang-template-api-echo",
    "golang_api_gin":"golang-template-api-gin"
}

In [58]:
class GoLangAPIApp(BaseModel):
    """Generate a GoLang API application type."""
    app_type: str = Field(
        ...,
        description="The type of GoLang API application to create. "
                    "Can be 'golang_api_echo' for Echo framework or 'golang_api_gin' for Gin framework. "
                    "If the user asks for 'golang api echo' or 'golang api dùng gin', "
    )

@tool("create_golang_api_app", args_schema=GoLangAPIApp)
def create_golang_api_app(app_type: str) -> str:
    print(app_type)
    return app_type


In [65]:
llm_tool = llm.bind_tools([create_golang_api_app])

In [63]:
response = llm.bind_tools([create_golang_api_app]).invoke([
    # {"role": "system", "content": "You are a specialized AI designed to interact solely with provided tools. If a user's request is not directly related to the functionality of the available tools, you must respond with: 'NOT within the scope of processing'"},
    {
    "role": "user",
    "content": "Create a Golang API Echo."
}
])

In [64]:
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rT1og68jxRV4XqpLYppjgWqk', 'function': {'arguments': '{\n  "app_type": "golang_api_echo"\n}', 'name': 'create_golang_api_app'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 112, 'total_tokens': 134, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BcmWM7BN2WSEp9RJnHlRWGtGQR2tI', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--34c13d48-a615-4321-8680-a4d32aa4a080-0', tool_calls=[{'name': 'create_golang_api_app', 'args': {'app_type': 'golang_api_echo'}, 'id': 'call_rT1og68jxRV4XqpLYppjgWqk', 'type': 'tool_call'}], usage_metadata={'input_tokens': 112, 'output_tokens': 22, 'total_tokens': 

In [67]:
class InitGitRepoState(TypedDict):
    messages: Annotated[list, add_messages]
    image_map: dict[str, str]

In [68]:
def get_app_feature(state: InitGitRepoState):
    response_tool = llm_tool.invoke(state["messages"])
    pass
